In [ ]:
import numpy as np
from collections import defaultdict
import json
import copy

In [ ]:
# all this +/- stuff is used for 
# * the 'current year MOV' prediction & contract value
# * projecting salary. 
def get_pm(ovr):
    iv = np.array([72.7 ,  7.28*2, 18.1/2 ])
    minVal = -2.021381747657189
    return minVal+iv[1]/(1+np.exp(-(ovr-iv[0])/iv[2]))
#plt.plot(np.arange(101),get_pm(np.arange(101)))
#plt.plot(np.arange(101),[o2m[i] for i in np.arange(101)])
#get_pm(50),get_pm(70)

# replacement level (average min contract signing)
RL = 45.542233117896146

# replacement level in MOV
r_lvl = get_pm(RL)

age_shift ={19: 6.6,
 20: 7.0,
 21: 5.1,
 22: 4.5,
 23: 2.4,
 24: 2.4,
 25: 0.2,
 26: 0.1,
 27: -0.9,
 28: -1.0,
 29: -1.6,
 30: -2.3,
 31: -3.4,
 32: -3.6,
 33: -3.7,
 34: -4.7,
 35: -5.1}
min_age_shift = max(age_shift.values())
max_shift = min(age_shift.values())


# expected mov weight for salary cap space, , weight for players, 

# win a round
weights = {
0 : ( [-2.849, -2.235, 0.544] ),
1 : ( [-1.723, 3.316, 0.368] ),
2 : ( [-1.225, 2.212, 0.217] ),
}
# draft value. (1) is pos -> %, (2) ovr,pot,age -> %, (3) pos -> ovr
draftP = np.array([0.28742864, 0.28181499, 0.61438846])
def winp_draft(ovr,pot,age):
    #xv = 4.3341 + ovr*0.1294 + pot*0.0343 + age*(-0.7099)
    xv = 3.606 + ovr*0.1271 + pot*0.0381 + age*(-0.6824)
    return 1/(1+np.exp(-xv))
draftOVR= np.array([ -0.07936123,  21.17839312, -28.19483771])

# team value
team_mov = [-0.21133543900273594, 0.35989254239441687, 96.76582740557117]

# salary multiplier
sA = 3.8

In [ ]:
age_shift_int = defaultdict(int)
age_min,age_max = min(age_shift.keys()),max(age_shift.keys())
for draft_age in range(17,27):
    for age in range(draft_age,age_max):
        total = 0
        seen = 0
        for i in range(draft_age,age_max):
            prog = age_shift.get(i,min_age_shift)
            if prog > 0:
                total += prog
        for i in range(draft_age,age):
            prog = age_shift.get(i,min_age_shift)
            if prog > 0:
                seen += prog
        if total > 0:
            frac = 1.0-seen/total
            if frac > 0:
                age_shift_int[(draft_age,age)] = frac
age_shift_int

In [ ]:
team = { 'p' : [(22,66,22000,0,19,50,70,4),(22,66,22000,1,19,50,70,1),(22,66,22000,2,19,50,70,2),(22,66,22000,3,19,50,70,3)], #(age,ovr,salary,years_left,draft_age,draft_ovr,draft_pot,draft_pick)
         'd' : [(1, 0, 8.6),(1, 0, -5.35),(2,0,3),(1,1,3),(2,1,3),(1,2,3),(2,2,3),(1,3,3),(2,3,3),(1,4,3),(2,4,3)] #(round,years_left,team_MOV)
       }
YEARS_TO_MODEL = 3
DISCOUNT = 1.0 # how much of a hometown discount do players get
                # otherwise bad teams give up good players in trades too easy
                # because they think they'll want an absurd contract & prefer some value

def eval_state(pars,tss,sCap,minS):
    pred_win_p = []
    for i in range(YEARS_TO_MODEL):
        play = [p for p in pars[i] if p >= RL]
        lp = len(play)
        if lp < 10:
            play= play + (10-lp)*[RL]
        play = sorted(play,reverse=True)[:10]
        play_s = sum([np.exp(i*team_mov[0])*p for i,p in enumerate(play)])*team_mov[1] -team_mov[2]

        cap_hit = tss[i] + (10-lp)*minS 

        diff = (sCap-cap_hit)/sCap
        cap_space = np.maximum(diff,0.1*diff)
        x = weights.get(i,weights[2])

        #p_mov = play_s + (x[0] + x[1]*cap_space)
        #win_p = 1.0/(1+np.exp(-(p_mov*x[3]+x[2])))
        
        ppow = x[0] + x[1]*cap_space + x[2]*play_s
        win_p = 1.0/(1+np.exp(-ppow))
        
        pred_win_p.append(win_p)

    # discount factor for the future, more uncertainty, less sure reward
    #value = [wp*(0.9**(i)) for i,wp in enumerate(win_p)]
    return pred_win_p


def get_team_value(team,sCap=90000,minS=750,teamNum=30):
   
    # turn mov into draft pick and future mov
    m2pos = lambda x: int(round(np.clip( (teamNum-1)/(1+np.exp(0.0048 - 0.4037*(x))),0,teamNum-1)))
    m2next = lambda year,mov: [1,0.5,0.25,0.08,0.03,0.01,0.01,0.01,0.01,0.01][year]*mov 

    # turn draft picks into specific predictions
    draft_picks = [ (yr,m2pos(m2next(yr,mov))+teamNum*(rnd-1)) for rnd,yr,mov in team['d']]
    
    pars = defaultdict(list) # player value
    tss = defaultdict(int)  # contracts
    dpars = [] # draft value

    # analyze existing contracts
    for age,ovr,con,yrl,draft_age,draft_ovr,draft_pot,draft_pick in team['p']:
        ovr2  = ovr
        povrs = [ovr2]
        
        # compute aging value
        for i in range(max(YEARS_TO_MODEL,yrl)):
            ovr2+=age_shift.get(age+i,max_shift)
            povrs.append(ovr2)
            
        # this is pretty good +/-
        pmovs = [get_pm(ovr2) for ovr2 in povrs]
        # this is ... okay estimate of fair contract
        # assumption: max contract is 1/3 salary cap
        ccont = [(sCap/3)*min(1,(pmov-r_lvl)/sA) for pmov in pmovs]
        # how much value do we get each contract year
        cvals = [c-con for c in ccont]

        # add existing contract
        for i in range(yrl+1):
            pars[i].append(povrs[i])
            tss[i] += con
            
        # extend contract (at fair price)
        if yrl+1 < YEARS_TO_MODEL:
            prev_val = eval_state(pars,tss,sCap,minS)

            tss2 = copy.deepcopy(tss)
            pars2 = copy.deepcopy(pars)

            for i in range(yrl+1,YEARS_TO_MODEL):
                pars2[i].append(povrs[i])
                tss2[i] += max(DISCOUNT*ccont[i],minS)
                
            # only if extending is the right thing
            if eval_state(pars2,tss2,sCap,minS) > prev_val:
                pars = pars2
                tss = tss2
        
        # add excess value into 3 years
        if yrl+1 > YEARS_TO_MODEL:
            amount_to_add = sum(cvals[YEARS_TO_MODEL:])/YEARS_TO_MODEL
            for i  in range(YEARS_TO_MODEL):
                tss[i] -= amount_to_add
    
    # compute draft pick value
    for yr,p in draft_picks:
        dpars.append((0.95**yr)*draftP[1]*np.exp(-draftP[0]*p**draftP[2]))
    
    # add draft picks to roster
    # make them zero cost
    for yr,p in draft_picks:
        if yr+1 < YEARS_TO_MODEL:
            ovr = np.exp(draftOVR[0]*p)*draftOVR[1]-draftOVR[2]
            for i in range(yr+1,YEARS_TO_MODEL):
                ovr = ovr + age_shift[20-(yr+1)+i]
                pars[i].append(ovr)
    
    # add young player value into long-term estimate
    for age,ovr,con,yrl,draft_age,draft_ovr,draft_pot,draft_pick in team['p']:
        # for real_rosters draft ratings aren't available!
        # dpars.append(age_shift_int.get((draft_age,age),0)*winp_draft(draft_ovr,draft_pot,draft_age))
        dpars.append(age_shift_int.get((draft_age,age),0)* (draftP[1]*np.exp(-draftP[0]*draft_pick**draftP[2])))

    value = eval_state(pars,tss,sCap,minS)
    #print(pred_movs,win_p,sum(value),sum(dpars))
    return sum(value) + sum(dpars)
get_team_value(team,90000,750)

In [ ]:
data = json.load(open('real_2020.json','rt',encoding='utf-8-sig')) #real_2020
gA = {_['key']:_['value'] for _ in data['gameAttributes']}

In [ ]:
team_players = defaultdict(list)
team_names = {}
season = gA['season']
sCap = gA['salaryCap']
minS = gA['minContract']
maxS = gA['maxContract']

team_injuries = defaultdict(list)
for p in data['players']:
    #draft_age,draft_ovr,draft_pot
    if p['tid'] >= 0:
        ovr = p['ratings'][-1]['ovr']
        age = season - p['born']['year']
        salary = p['contract']['amount']
        years_left = p['contract']['exp']-season
        dp = p['draft']['pick'] + 30 * (p['draft']['round']-1)
        if dp < 0:
            dp = 61
        team_players[p['tid']].append((age,ovr,salary,years_left,p['draft']['year']-p['born']['year'],p['draft']['ovr'],p['draft']['pot'],dp))
        team_injuries[p['tid']].append(p['injury']['gamesRemaining'])

In [ ]:
team_movs = defaultdict(float)

for t in data['teams']:
    tid = t['tid']
    team_names[t['tid']] = t['abbrev']
    for ts in t['stats']:
        if ts['playoffs']:
            continue
        current_mov = 0
        gp = ts['gp']+1e-9
        gl = 82-gp+1e-9
        if season == ts['season'] and not ts['playoffs'] and ts['gp']>0:
            mov = (ts['pts'] - ts['oppPts']) / ts['gp'];
            current_mov = mov
        estimated_mov = sum(sorted([get_pm(_[1])*(max(gl-i,0)/gl) for _,i in zip(team_players[tid],team_injuries[tid])])[-10:])
        team_movs[tid] = (gp/82)*current_mov + (gl/82)*estimated_mov    

In [ ]:
team_picks = defaultdict(list)
for d in data['draftPicks']:
    mov = team_movs[d['originalTid']]
    tid = d['tid']
    rnd = d['round']
    yl = d['season']-season
    team_picks[tid].append((rnd,yl,mov))

In [ ]:
teams_vals = []
for i in range(len(data['teams'])):
    print(team_names[i])
    val = get_team_value({'mov':team_movs[i],'p':team_players[i],'d':team_picks[i]},sCap,minS)
    teams_vals.append((val,team_names[i]))

In [ ]:
for v,t in sorted(teams_vals)[::-1]:
    print(round(100*v,2),t)

In [ ]:
import random
deals = []
for i in range(100):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1], 'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2], 'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    pn = np.random.randint(len(team_picks[t1]))
    picks = [_ for _ in team_picks[t1]]
    pick = picks[pn]        
    del picks[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':team_players[t1] + [player],'d':picks},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p,'d':team_picks[t2] + [pick]},sCap,minS)
        if t1v > t1vo and t2v > t2vo:
            val = min((t1v-t1vo),(t2v-t2vo))
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [ ]:
sorted(deals,reverse=True)

In [ ]:
import random
deals = []
for i in range(100):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1],'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2],'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1]},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p + [pick],'d':team_picks[t2]},sCap,minS)
        if t1v > t1vo and t2v > t2vo:
            val = (t1v-t1vo) + (t2v-t2vo)
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [ ]:
for d in sorted(deals,reverse=True):
    if d[4][2]/d[2][2] < 1.25 and d[4][2]/d[2][2] > 1/1.25:
        print(d)

In [ ]:
import random
deals = set()
for i in range(100):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1],'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2],'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        if not (pick[2]/player[2] < 1.25 and pick[2]/player[2] > 1/1.25):
            continue
        for p2 in range(len(team_picks[t2])):
            local_picks = [_ for _ in team_picks[t2]]
            pick2 = local_picks[p2]       
            del local_picks[p2]
            t1v1 = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1]},sCap,minS)

            t1v = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1] + [pick2]},sCap,minS)
            t2v = get_team_value({'mov':team_movs[t2],'p':local_p + [pick],'d':local_picks },sCap,minS)
            if t1v1 < t1vo and t1v > t1vo and t2v > t2vo:
                v1 = (t1v-t1vo)
                v2 = (t2v-t2vo)
                val = min( v1 , v2)
                deals.add((val,v1,v2,team_names[t1],pick,team_names[t2],player,pick2))
                

In [ ]:
sorted(list(deals),reverse=True)

In [ ]:
team_names

In [ ]:
team_test = {'mov':team_movs[2],'d':team_picks[2],'p':team_players[2]}
get_team_value(team_test,sCap,minS)

In [ ]:
team_test = {'mov':team_movs[29],'d':team_picks[29],'p':team_players[29]}
get_team_value(team_test,sCap,minS)


team_test = {'mov':team_movs[3],'d':team_picks[3],'p':team_players[3]}
get_team_value(team_test,sCap,minS)

In [ ]:
team_test ={'mov': -2.97562361535948,
 'd': [(2, 0, 3.1711893144802534),
  (1, 0, -2.97562361535948),
  (2, 0, -1.0163612554249681),
  (2, 1, 3.1711893144802534),
  (1, 1, -2.97562361535948),
  (2, 1, 14.450161462168422),
  (1, 2, -2.97562361535948),
  (2, 2, -2.97562361535948),
  (1, 3, -2.97562361535948),
  (1, 4, -2.97562361535948),
  (2, 4, -2.97562361535948),
  (1, 5, -2.97562361535948),
  (2, 5, -2.97562361535948),
  (1, 6, -2.97562361535948),
  (2, 6, -2.97562361535948)],
 'p': [(24, 55, 1200, 2),
  (26, 56, 19900, 2),
  (27, 55, 14500, 1),
  (27, 44, 17000, 0),
  (21, 56, 3800, 3),
  (24, 38, 900, 0),
  (23, 43, 900, 0),
  (22, 42, 900, 1),
  (22, 55, 3750, 2),
  (25, 55, 1400, 3),
  (24, 34, 900, 0),
  (31, 50, 25550, 1),
  (22, 53, 4030, 1),
  (26, 48, 1550, 0),
  (22, 38, 900, 0),
  (24, 51, 1600, 0)]}
get_team_value(team_test,sCap,minS)